In [423]:
# autoreload
%load_ext autoreload
%autoreload 2

# fix path
import sys
sys.path.append("/home/jxm/reverse-training")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [424]:
from utils.analysis import load_results_as_df
import os


HOME_DIR = "/home/jxm/reverse-training"
df = load_results_as_df(os.path.join(HOME_DIR, "results"))
print("Length:", len(df))

df = df[~df["dataset"].str.contains("newsgroup")]


Filtered 0/375 results with no evaluation metrics
dict_keys(['data', 'full_ot_distance', 'sinkhorn_distance', 'jaccard_overlap_examples', 'jaccard_overlap_vocabulary', 'levenshtein_stats', 'args', 'expert_evaluation_metrics', 'evaluation_metrics'])
Length: 375


In [425]:
df["dataset"].value_counts()

dataset
ag_news_10000    142
dbpedia_10000    138
Name: count, dtype: int64

In [426]:
[col for col in df.columns if "acc" in col]

['expert_evaluation_best_eval_accuracy',
 'evaluation_best_eval_accuracy',
 'dataset_jaccard_overlap_examples',
 'dataset_jaccard_overlap_vocabulary']

In [427]:
import pandas as pd

pd.set_option('display.max_rows', None)

dv_cols = ["dataset", "select_seed_dataset", "select_full_dataset_size", "select_batch_fill_strategy"]

iv_cols = ["evaluation_best_eval_accuracy", "dataset_sinkhorn_distance", "dataset_jaccard_overlap_vocabulary"]

# final_df = df[df["select_seed_dataset"] == "nq_500000"]

df.groupby(dv_cols)[iv_cols].mean().sort_values(
    by=["dataset","select_seed_dataset", "select_full_dataset_size", "select_batch_fill_strategy"], 
    ascending=False
)

evaluation_best_eval_accuracy  \
dataset       select_seed_dataset select_full_dataset_size select_batch_fill_strategy                                  
dbpedia_10000 nq_100000           2500                     topk_balanced                                    0.527832   
                                                           topk                                             0.136393   
                                                           random                                           0.507487   
                                                           greedy_batched                                   0.635905   
                                                           bottomk                                          0.137695   
                                  1000                     topk_balanced                                    0.461914   
                                                           topk                                             0.100098   
                                                           random                                           0.416504   
                                                           greedy_batched                                   0.500326   
                                                           greedy                                           0.638184   
                                                           bottomk                                          0.102376   
                                  500                      topk_balanced                                    0.379883   
                                                           topk                                             0.079264   
                                                           random                                           0.312988   
                                                           greedy_batched                                   0.401367   
                                                           greedy                                           0.479818   
                                                           bottomk                                          0.075195   
                                  100                      topk_balanced                                    0.163900   
                                                           topk                                             0.045410   
                                                           random                                           0.149414   
                                                           greedy_batched                                   0.161133   
                                                           greedy                                           0.134440   
                                                           bottomk                                          0.045410   
              nq_10000            2500                     topk_balanced                                    0.706055   
                                                           topk                                             0.409017   
                                                           random                                           0.655762   
                                                           greedy_batched                                   0.702962   
                                                           greedy                                           0.691406   
                                                           bottomk                                          0.397135   
                                  1000                     topk_balanced                                    0.593099   
                                                           topk                                             0.202148   
                                                           random                                           0.550293   
       

In [428]:


dv_cols = ["dataset", "select_seed_dataset", "select_full_dataset_size", "select_batch_fill_strategy"]

iv_cols = ["evaluation_best_eval_accuracy", "dataset_sinkhorn_distance", "dataset_levenshtein_stats_average_distance", "dataset_full_ot_distance"]


# Final results: dataset size 1000, batch fill strategy is not bottomk
final_df = df[
    (df["select_full_dataset_size"] == 1000) & 
    (df["select_batch_fill_strategy"] != "bottomk") &
    (df["dataset"] != "newsgroup_10000")
].reset_index()
final_df = final_df.groupby(dv_cols)[iv_cols].mean().sort_values(
    by=["dataset","select_seed_dataset", "select_full_dataset_size", "select_batch_fill_strategy"], 
    ascending=False
)
final_df = final_df.reset_index()
final_df = final_df.pivot(index=["select_seed_dataset", "select_batch_fill_strategy"], columns=["dataset"], values=iv_cols)
final_df = final_df.swaplevel(0, 1, axis=1)
final_df = final_df.sort_index(axis=1)
final_df

dataset                                                   ag_news_10000  \
                                               dataset_full_ot_distance   
select_seed_dataset select_batch_fill_strategy                            
nq_10000            greedy                                     1.022224   
                    greedy_batched                             1.031292   
                    random                                     1.055060   
                    topk                                       1.052283   
                    topk_balanced                              1.050121   
nq_100000           greedy                                     1.019220   
                    greedy_batched                             1.043698   
                    random                                     1.052631   
                    topk                                       1.050810   
                    topk_balanced                              1.050741   

dataset                                                                                    \
                                               dataset_levenshtein_stats_average_distance   
select_seed_dataset select_batch_fill_strategy                                              
nq_10000            greedy                                                     168.423900   
                    greedy_batched                                             168.459733   
                    random                                                     168.475533   
                    topk                                                       168.629933   
                    topk_balanced                                              168.633700   
nq_100000           greedy                                                     168.237767   
                    greedy_batched                                             168.560367   
                    random                                                     168.626267   
                    topk                                                       168.761233   
                    topk_balanced                                              168.745600   

dataset                                                                   \
                                               dataset_sinkhorn_distance   
select_seed_dataset select_batch_fill_strategy                             
nq_10000            greedy                                      1.266004   
                    greedy_batched                              1.273789   
                    random                                      1.292871   
                    topk                                        1.288595   
                    topk_balanced                               1.288923   
nq_100000           greedy                                      1.261858   
                    greedy_batched                              1.283533   
                    random                                      1.291164   
                    topk                                        1.286754   
                    topk_balanced                               1.288560   

dataset                                                                       \
                                               evaluation_best_eval_accuracy   
select_seed_dataset select_batch_fill_strategy                                 
nq_10000            greedy                                          0.800618   
                    greedy_batched                                  0.753255   
                    random                                          0.668294   
                    topk                                            0.427246   
                    topk_balanced                                   0.681152   
nq_100000           greedy                                          0.777832   
                    greedy_batched                                  0.677246   
                    rando

In [429]:
final_df[[col for col in final_df.columns if ("acc" in col[1])]]

dataset                                                        ag_news_10000  \
                                               evaluation_best_eval_accuracy   
select_seed_dataset select_batch_fill_strategy                                 
nq_10000            greedy                                          0.800618   
                    greedy_batched                                  0.753255   
                    random                                          0.668294   
                    topk                                            0.427246   
                    topk_balanced                                   0.681152   
nq_100000           greedy                                          0.777832   
                    greedy_batched                                  0.677246   
                    random                                          0.670410   
                    topk                                            0.380046   
                    topk_balanced                                   0.597493   

dataset                                                        dbpedia_10000  
                                               evaluation_best_eval_accuracy  
select_seed_dataset select_batch_fill_strategy                                
nq_10000            greedy                                          0.647298  
                    greedy_batched                                  0.638997  
                    random                                          0.550293  
                    topk                                            0.202148  
                    topk_balanced                                   0.593099  
nq_100000           greedy                                          0.638184  
                    greedy_batched                                  0.500326  
                    random                                          0.416504  
                    topk                                            0.100098  
                    topk_balanced                                   0.461914

In [430]:
print(final_df.to_latex(float_format=lambda x: f"{100*x:.1f}", escape=True))

\begin{tabular}{llrrrrrrrr}
\toprule
 & dataset & \multicolumn{4}{r}{ag\_news\_10000} & \multicolumn{4}{r}{dbpedia\_10000} \\
 &  & dataset\_full\_ot\_distance & dataset\_levenshtein\_stats\_average\_distance & dataset\_sinkhorn\_distance & evaluation\_best\_eval\_accuracy & dataset\_full\_ot\_distance & dataset\_levenshtein\_stats\_average\_distance & dataset\_sinkhorn\_distance & evaluation\_best\_eval\_accuracy \\
select_seed_dataset & select_batch_fill_strategy &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{5}{*}{nq\_10000} & greedy & 102.2 & 16842.4 & 126.6 & 80.1 & 91.9 & 22088.3 & 116.1 & 64.7 \\
 & greedy\_batched & 103.1 & 16846.0 & 127.4 & 75.3 & 92.7 & 22094.8 & 116.9 & 63.9 \\
 & random & 105.5 & 16847.6 & 129.3 & 66.8 & 96.7 & 22134.6 & 120.6 & 55.0 \\
 & topk & 105.2 & 16863.0 & 128.9 & 42.7 & 102.7 & 22231.7 & 124.7 & 20.2 \\
 & topk\_balanced & 105.0 & 16863.4 & 128.9 & 68.1 & 95.6 & 22087.5 & 119.4 & 59.3 \\
\cline{1-10}
\multirow[t]{5}{*}{nq\_100000} & greedy & 101.

In [431]:
df.columns

Index(['dataset', 'dataset_size', 'minibatch_size', 'eval_every',
       'max_iterations', 'max_iterations_x', 'pretrain_iterations_x',
       'lr_tokens', 'lr_lr', 'lr_teacher', 'syn_steps', 'seed', 'token_init',
       'expert_epochs', 'sequence_length', 'num_experts',
       'num_steps_per_expert', 'expert_batch_size', 'expert_lr',
       'discrete_optimizer', 'admm_penalty_term', 'gcg_search_width',
       'gcg_tokens_to_swap', 'gcg_documents_to_swap', 'select_seed_dataset',
       'select_projection_dim', 'select_steps_per_grad',
       'select_full_dataset_size', 'select_lr_student',
       'select_num_pseudoexperts', 'select_do_classification',
       'select_batch_fill_strategy', 'select_label_strategy',
       'select_do_warmup', 'select_grads_full_model', 'num_eval_epochs',
       'exp_name', '_wandb', 'expert_evaluation_best_eval_loss',
       'expert_evaluation_best_eval_accuracy', 'evaluation_best_eval_loss',
       'evaluation_best_eval_accuracy', 'dataset_sinkhorn_distan